<a href="https://colab.research.google.com/github/SeunghyeChae/CAKD3_tmdhey/blob/main/dl_%ED%95%A8%EC%88%98%ED%98%95API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sequential input 2개이상 output 2개이상에는 사용 못함 
# 각 층에서 가중치 계산
# optimizer에의해 손실함수 최소화시켜주기 위해 역전파 이용

# ex. input에 
# 데이터 설명하는 데이터 = 메타 데이터
# 사용자가 제공한 텍스트
# 제품 사진 
# 이런게 들어갈 경우 sequential어려움 

# ex. output에
# 장르 / 시대 등 ..


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras import Sequential,Model
from tensorflow.keras import layers
from tensorflow.keras import Input 

seq_model = Sequential()
seq_model.add(layers.Dense(32,activation='relu',input_shape=(64,)))
seq_model.add(layers.Dense(32,activation='relu'))
seq_model.add(layers.Dense(10,activation='softmax'))

seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
# 함수형 API로 입력텐서 셋팅해주고..
# 처음 인풋을 레이어에 넣어주고 *반복

In [5]:
input_tensor= Input(shape=(64,))
x= layers.Dense(32,activation='relu')(input_tensor)
x= layers.Dense(32,activation='relu')(x)
output_tensor= layers.Dense(10,activation='softmax')(x)

model= Model(input_tensor, output_tensor)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [8]:
import numpy as np
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

x_train = np.random.random((1000,64))
y_train= np.random.random((1000,10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train,y_train)

Epoch 1/10
8/8 [==============================] - 2s 3ms/step - loss: 11.8248
Epoch 2/10
8/8 [==============================] - 0s 3ms/step - loss: 12.7343
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 14.2153
Epoch 4/10
8/8 [==============================] - 0s 3ms/step - loss: 16.2699
Epoch 5/10
8/8 [==============================] - 0s 3ms/step - loss: 19.0647
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 22.5270
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 26.2051
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 30.2518
Epoch 9/10
8/8 [==============================] - 0s 3ms/step - loss: 34.3031
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 41.1593


In [9]:
score

41.1593132019043

# 다중 입력모델

In [15]:
# 다중 입력모델 

from tensorflow.keras import Sequential,Model
from tensorflow.keras import layers
from tensorflow.keras import Input 

text_vocabulary_size= 10000
question_vocabulary_size= 10000
answer_vocabulary_size = 500

text_input= Input(shape=(None,),
                  dtype='int32',
                  name='text')

embadded_text= layers.Embedding(
    text_vocabulary_size,64)(text_input)


encoded_text= layers.LSTM(32)(embadded_text)

question_input= Input(shape=(None,),
                  dtype='int32',
                  name='question')
embedded_question = layers.Embedding(
    question_vocabulary_size,32)(question_input)

encoded_question= layers.LSTM(32)(embedded_question)

concatenated= layers.concatenate([encoded_text,encoded_question],axis=-1)

answer= layers.Dense(answer_vocabulary_size,
                     activation='softmax')(concatenated)

model= Model([text_input, question_input],answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])


In [16]:
import numpy as np
text_vocabulary_size= 10000
question_vocabulary_size= 10000
answer_vocabulary_size=500
num_samples=1000
max_length= 100

text= np.random.randint(1,text_vocabulary_size,
                        size=(num_samples,max_length))
text.shape

(1000, 100)

In [17]:
from tensorflow.keras.utils import to_categorical

num_sample= 1000
max_length= 100

text= np.random.randint(1,text_vocabulary_size,
                        size=(num_samples,max_length))

question= np.random.randint(1,question_vocabulary_size,
                            size=(num_samples, max_length))

answers= np.random.randint(0,answer_vocabulary_size,size=num_samples)

answers = to_categorical(answers)

model.fit([text,question],answers,epochs=10,batch_size=128)

Epoch 1/10
8/8 [==============================] - 5s 15ms/step - loss: 6.2150 - acc: 0.0040
Epoch 2/10
8/8 [==============================] - 0s 15ms/step - loss: 6.1963 - acc: 0.0400
Epoch 3/10
8/8 [==============================] - 0s 14ms/step - loss: 6.1150 - acc: 0.0050
Epoch 4/10
8/8 [==============================] - 0s 14ms/step - loss: 6.0216 - acc: 0.0090
Epoch 5/10
8/8 [==============================] - 0s 14ms/step - loss: 5.9351 - acc: 0.0080
Epoch 6/10
8/8 [==============================] - 0s 14ms/step - loss: 5.8548 - acc: 0.0120
Epoch 7/10
8/8 [==============================] - 0s 14ms/step - loss: 5.7983 - acc: 0.0080
Epoch 8/10
8/8 [==============================] - 0s 14ms/step - loss: 5.7337 - acc: 0.0130
Epoch 9/10
8/8 [==============================] - 0s 13ms/step - loss: 5.6432 - acc: 0.0190
Epoch 10/10
8/8 [==============================] - 0s 14ms/step - loss: 5.5878 - acc: 0.0260


# 다중 출력모델

In [18]:
# [과제]다중 출력모델

# 층으로 구성된 비순환 유향 그래프

In [ ]:
# 잔차연결- resnet에서 했음 

In [ ]:
# from tensorflow.keras import layers

# branch_a= layers.Conv2D(128,1,
#                         activation='relu',strides=2)(x)

# branch_b= layers.Conv2D(128,1,activation='relu')(x)
# branch_b= layers.Conv2D(128,1,activation='relu',strides=2)(branch_b)

# branch_c= layers.AveragePooling2D(3,strides=2)(x)
# branch_c= layers.Conv2D(128,1,activation='relu')(branch_c)